In [ ]:
'''
    此处约定参数名及其含义
    frequent_list：所有已确定频繁的项集（已满足先验条件）
        使用list[]来存储，存储着频繁项集frozenset()，
    candidate_list：候选项集，来源于频繁项集的连接与剪枝
        使用list[]来存储，内含项集[]
    count_dict：临时存放该候选项出现的次数
    所有数据项以字符串的形式存储（数字型可通过char()来转化为数字）
    所有事务使用列表存储，列表中存储数据项
'''

In [19]:
#初始化测试数据
MIN_SUPPORT = 0.5
MIN_CONFIDENCE = 0.6
DEFAULT_DATASET = [['1', '2', '13', '14'], ['1', '3', '13'], ['1', '2', '13']]

In [20]:
#处理候选项集，舍去非频繁项集
def frozen_candidate_list(can_dict):
    frozen_list = []
#     print(can_dict)
    for candidate_item in list(can_dict):
#         print(candidate_item)
        if candidate_item not in support_list:
            if type(candidate_item).__name__ == 'frozenset':
                support_list[candidate_item] = can_dict[candidate_item]
            else:
                support_list[frozenset({candidate_item})] = can_dict[candidate_item]
        if can_dict[candidate_item] / COUNT_TASK < MIN_SUPPORT:
            can_dict.pop(candidate_item)
        else:
#             print(frozenset({candidate_item}))
            if type(candidate_item).__name__ == 'frozenset':
                frozen_list.append(candidate_item)
            else:
                frozen_list.append(frozenset({candidate_item}))
            
    return frozen_list

In [21]:
#组合两集合
def join_item(set1, set2):
    if len(set1) != len(set2):
#         print('长度不相同 请检查数据')
        return False
    if len(set1 - set2) != 1:
#         print('组合失败')
        return False
#     print('组合',set1,'与',set2,'为：',set1 | set2)
    return set1 | set2

In [22]:
#检查是否应该剪枝
def cut_item(candidate_item):
#     print('接收到需检查子集是否频繁的候选项：',candidate_item)
    #此时candidate_item已是frozenset形式，需检查其子串是否频繁
#     print('开始获取子串')
    
    item_list = list(candidate_item)
#     for item in candidate_item:
#         item_list.append(list(item))
    
    #获取到的是frozenset
#     print('获取item_list：',item_list)
    
    #temp作为判断位数，不取第temp位的数
    for temp in range(len(item_list)):
#         print('不取第',temp,'位数')
        temp_list = []
        #获取子串
        for index in range(len(item_list)):
            if index != temp:
                temp_list.append(item_list[index])
                
#         print('获取到临时子串列表：',temp_list)
        #获取到frozenset形式的子串
        temp_frozenset = frozenset(temp_list)
#         print('临时子串列表转换为frozenset：',temp_frozenset)
#         print('candidate_list=',candidate_list)
#         print('len(candidate_item)',len(candidate_item))
#         print('candidate_list[0]', candidate_list[0])
#         print('###########candidate_item=',candidate_item)
#         print('###########len(candidate_item)=',len(candidate_item))
#         print('###########candidate_list=',candidate_list)
        if temp_frozenset not in candidate_list[len(candidate_item)-2]:
#         if temp_frozenset not in last_fre_dict:
#             print('检查到：',temp_frozenset,'不在',candidate_list[len(candidate_item)-2],'中 ——> 应该剪枝')
            return True
#         print('检查到：',temp_frozenset,'在',candidate_list[len(candidate_item)-2],'中 ——> 不应该剪枝')
    return False

In [23]:
#传入新长度的频繁项集，对该项集进行处理，项集为列表，内含多个等长频繁项集
def candidate_gen(candidate_item):
    
    if len(candidate_item) == 0:
        return
#     print('检测长度为',len(candidate_item[0]),'的项集：',candidate_item)
    new_len_list = []
    candidate_dict = {}
    last_fre_dict = {}
    
    for frozen_set1 in candidate_item:
#         print('左候选项')
#         print(frozen_set1)
        for frozen_set2 in candidate_item:
#             print('右候选项')
#             print(frozen_set2)
            
            if frozen_set1 == frozen_set2:
#                 print('左右候选项相同，跳过')
                continue
            
            count = 0
            
            new_set = join_item(frozen_set1, frozen_set2)
#             print('开始计数',new_set,'的支持度计数')
            if new_set == False:
                continue
            if new_set not in candidate_dict:
                for tasks in DEFAULT_DATASET:
                    if new_set<frozenset(tasks):
                        count += 1
                candidate_dict[new_set] = count
#                 print('添加',new_set,'的支持度计数为：',count)
#             else:
#                 print('已有',new_set,'的支持度计数为：',candidate_dict[new_set])
#         print()
        
#     print('已获取新组合候选项集：',candidate_dict)

    candidate_dict = frozen_candidate_list(candidate_dict)
#     print('获取到长度为',len(candidate_item)-1,'的频繁候选项集：',candidate_dict)
#     print('进行剪枝')
    candidate_dict_list = list(candidate_dict)
#     print('#########candidate_dict_list=',candidate_dict_list)
    for candidate_item in candidate_dict_list:
#         print('检查',candidate_item,'是否有非频繁子集')
        if cut_item(candidate_item):
            candidate_dict_list.pop(candidate_item)
#     candidate_dict = frozenset(candidate_dict_list)
#     new_len_list.append(candidate_dict_list)
    
#     print('结束剪枝')
#     print('最终得到频繁项集：',candidate_dict_list)
    candidate_list.append(candidate_dict_list)
    
#     print('新候选频繁项集：',candidate_list)
#     print()
    
    candidate_gen(candidate_list[len(candidate_list)-1])

In [24]:
#主函数
import time
if __name__ == '__main__':
    
    support_list = {}
    
    start_time = time.time()
    
    datafile = open('D:/jupyterWorkSpace/mushroom.dat')
    DEFAULT_DATASET = []
    for line in datafile.readlines():
        datalist = line.split(' ')
        datalist[len(datalist)-1] = datalist[len(datalist)-1][:len(datalist[len(datalist)-1])-1]
        DEFAULT_DATASET.append(datalist)
#     print(DEFAULT_DATASET)
    
    DATASET = []
    #存放已确定频繁项集（从candidate_list中取）
    frequent_list = []
    #存放候选列表，包含所有或频繁项集，项集以tuple或frozenset的形式存放
    candidate_list = []
    #存放候选项，及其对应出现次数
    candidate_dict = {}
    #上一频繁项集
    last_fre_dict = {}
    #存放总事务数
    COUNT_TASK = 0
    
    
    for tasks in DEFAULT_DATASET:
        DATASET.append(frozenset(tasks))
#         print(DATASET)
        for item in tasks:
            if item not in candidate_dict:
                candidate_dict[item] = 1
#                 print('添加后')
#                 print(candidate_dict)
            else:
#                 print('增加后')
#                 print(candidate_dict)
                candidate_dict[item] += 1
        COUNT_TASK += 1
    
#     print('总事务数：',COUNT_TASK)
    
    #将候选项集字典中的频繁项转为frozenset，存入候选列表中
    candidate_list.append(frozen_candidate_list(candidate_dict))
#     print('最终获得有初始频繁项集：',candidate_list)
#     print(candidate_list[0])
    
    print()
    
    #开始递推生成新项集
    candidate_gen(candidate_list[0])
    
    for index in range(len(candidate_list[len(candidate_list)-2])):
#         print(candidate_list[len(candidate_list)-2][index])
        candidate_list[len(candidate_list)-2][index] = list(candidate_list[len(candidate_list)-2][index])
        candidate_list[len(candidate_list)-2][index].sort()
    
    print('最终最长项集为：', candidate_list[len(candidate_list)-2] )
    
    print('最终用时：',time.time()-start_time)
    
    print('检验强关联规则:')
    
#     print(support_list)
    
    for item in candidate_list[len(candidate_list)-2]:
        for len_of_item in range(0, len(item)//2):
            for start in range(len(item) - len_of_item):
#                 print(item[start : start + len_of_item + 1])
                con_A = frozenset(item[start : start + len_of_item + 1])
                con_B = frozenset(item[:start] + item[start + len_of_item + 1 : len(item)+1])

#                 print('confidence( %s => %s ) = %f' %(con_A, con_B, support_list[con_A] / support_list[con_B])
#                 print('confidence( %s => %s ) = %f' %(con_B, con_A, support_list[con_B] / support_list[con_A])
#                 print(item)
#                 print(con_A)
#                 print(support_list[con_A])
#                 print(con_B)
#                 print(support_list[con_B])
                
                print('confidence(', list(con_A), '->', list(con_B), ')= ', support_list[frozenset(item)], '/', support_list[con_A], '=', support_list[frozenset(item)] / support_list[con_A])
                print('confidence(', list(con_B), '->', list(con_A), ')= ', support_list[frozenset(item)], '/', support_list[con_B], '=', support_list[frozenset(item)] / support_list[con_B])
            
        
                
    
#     print(support_list)
    
#     frequent_list.append(candidate_gen(candidate_list))
    
#     frequent_list = []
#     candidate_list = []
#     count_dict = {}
#     COUNT_TASK = 0
    
#     print(DEFAULT_DATASET)
    
#     #获取一次项集
#     for tasks in DEFAULT_DATASET:
#         for item in tasks:
#             if item not in count_dict:
#                 count_dict[item] = 1
#             else:
#                 count_dict[item] += 1
#         COUNT_TASK += 1
    
#     print('初始项集')
#     print(count_dict)
    
#     for key in list(count_dict):
#         if count_dict[key] / COUNT_TASK < MIN_SUPPORT:
#             count_dict.pop(key)
            
#     print('初始频繁项集')
#     print(count_dict)
#     candidate_list.append(frozenset(count_dict))
#     print('整合频繁项集')
#     frequent_list.append(candidate_list)
#     print(frequent_list)
    
    
#     #获取单个事务
#     for task in DEFAULT_DATASET:
#         #获取事务中的各项
#         for candidate in task:
# #             print(candidate)
#             if candidate not in candidate_list:
#                 candidate_list.append(candidate)
#                 print('添加:')
#                 print(candidate_list)
#             else:
#                 candidate_list[candidate] += 1
#                 print('增加:')
#                 print(candidate_list)
#         COUNT_TASK += 1
    
#     for key in list(candidate_list):
#         if candidate_list[key]/COUNT_TASK < MIN_SUPPORT:
#             candidate_list.pop(key)
#     print('总事务数有：' + str(COUNT_TASK))
#     frequent_list.append(candidate_list)
#     print(frequent_list)
    
    


最终最长项集为： [['34', '36', '59', '85', '86'], ['34', '36', '85', '86', '90'], ['34', '36', '39', '85', '86'], ['34', '59', '85', '86', '90'], ['34', '63', '85', '86', '90'], ['34', '39', '85', '86', '90'], ['24', '34', '85', '86', '90'], ['34', '53', '85', '86', '90']]
最终用时： 4.878932952880859
检验强关联规则:
confidence( ['34'] -> ['36', '85', '59', '86'] )=  4232 / 7914 = 0.5347485468789487
confidence( ['36', '85', '59', '86'] -> ['34'] )=  4232 / 4232 = 1.0
confidence( ['36'] -> ['59', '34', '85', '86'] )=  4232 / 6812 = 0.6212566059894304
confidence( ['59', '34', '85', '86'] -> ['36'] )=  4232 / 4984 = 0.8491171749598716
confidence( ['59'] -> ['36', '34', '85', '86'] )=  4232 / 5176 = 0.8176197836166924
confidence( ['36', '34', '85', '86'] -> ['59'] )=  4232 / 6602 = 0.6410178733717056
confidence( ['85'] -> ['36', '34', '59', '86'] )=  4232 / 8124 = 0.5209256523879862
confidence( ['36', '34', '59', '86'] -> ['85'] )=  4232 / 4232 = 1.0
confidence( ['86'] -> ['36', '34', '85', '59'] )=  4232 / 

In [7]:
# set1 = frozenset({'1':1,'3':3,'4':4})
# print(set1)
set2 = frozenset({'3':3,'4':4,'5':5,'6':6,'13':3})
print(set2)
# set3 = frozenset(list(set1) + list(set2))
# print(set3)
dict1 = {set2:1}
print(dict1)

# set4 = set2 - set1
# print(set4)

test_list = []
for item in list(list(dict1)[0]):
    print(item)
    test_list.append(frozenset({item}))
print(test_list)

# for item in set2:
#     print(item)
#     if item in set2:
#         print('true')
        


frozenset({'4', '5', '13', '3', '6'})
{frozenset({'4', '5', '13', '3', '6'}): 1}
4
5
13
3
6
[frozenset({'4'}), frozenset({'5'}), frozenset({'13'}), frozenset({'3'}), frozenset({'6'})]


In [225]:
set1 = frozenset('1')
set2 = frozenset('2')
print(len(set1))

list1 = [1,2]
print(set1.issubset(list1))
print(list1)

1
False
[1, 2]


In [13]:
# list1 = [frozenset({'1','4','6'}), frozenset({'3'})]
# frozen_set1 = frozenset({'1','6','4'})
# for item in frozen_set1:
#     print(item)
    
# list2 = [1,4,6,13] 
# print(frozenset(list2))

# list3 = [frozenset({'1','3'})]
# list4 = frozenset({1,3})
# print(list3)
# print(list4)

# print(list(list3))

# print(type(list4))
# print(type(list4).__name__ == 'frozenset')

# list1 = [1,34,23,13,43,54,2,25]
# list1.sort()
# print(list1)
# del list

# print(4//2)
# print(5//2)
# print(6//2)

fset1 = frozenset('1')
print('%s %s' %(fset1,'test2'))

frozenset({'1'}) test2
